<a href="https://colab.research.google.com/github/alscop/ESAA-25-2/blob/main/ESAA_YB_code_review_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

수상작 리뷰
(https://dacon.io/competitions/official/235959/codeshare/6477?page=1&dtype=recent)

# 주제 및 데이터
여행 상품 신청 여부 예측 AI 해커톤  
: 여행사들이 고객 데이터를 이용해 여행 패키지의 수요를 미리 예측할 수 있도록 함.


데이터
- id : 샘플 아이디
- Age : 나이
- TypeofContact : 고객의 제품 인지 방법 (회사의 홍보 or 스스로 검색)
- CityTier : 주거 중인 도시의 등급. (인구, 시설, 생활 수준 기준) (1등급 > 2등급 > 3등급)
- DurationOfPitch : 영업 사원이 고객에게 제공하는 프레젠테이션 기간
- Occupation : 직업
- Gender : 성별
- NumberOfPersonVisiting : 고객과 함께 여행을 계획 중인 총 인원
- NumberOfFollowups : 영업 사원의 프레젠테이션 후 이루어진 후속 조치 수
- ProductPitched : 영업 사원이 제시한 상품
- PreferredPropertyStar : 선호 호텔 숙박업소 등급
- MaritalStatus : 결혼여부
- NumberOfTrips : 평균 연간 여행 횟수
- Passport : 여권 보유 여부 (0: 없음, 1: 있음)
- PitchSatisfactionScore : 영업 사원의 프레젠테이션 만족도
- OwnCar : 자동차 보유 여부 (0: 없음, 1: 있음)
- NumberOfChildrenVisiting : 함께 여행을 계획 중인 5세 미만의 어린이 수
- Designation : (직업의) 직급
- MonthlyIncome : 월 급여
- ProdTaken : 여행 패키지 신청 여부 (0: 신청 안 함, 1: 신청함)

# 코드 리뷰 : 각 분석 과정의 코드 흐름을 요약 정리(EDA, 전처리, 모델링 등)

EDA
- 박스플롯으로 이상치 확인.(IQR 활용)




전처리
- 결측치 확인: 칼럼별 결측치 개수 계산하는 함수 제작, 이용.
- 인코딩: 결측치 유지 위해 결측치 없는 칼럼과 있는 칼럼을 나누어 인코딩. - 이때 train을 먼저 인코딩해서 transform을 시키면 이후 test 인코딩 할때 fit을 못하기 때문에 test를 먼저 인코딩 해줌.
- 스케일링: tree기반의 catboost를 사용하기 때문에 스케일링의 영향은 거의 없을 것이라 생각했지만, 결측치를 채우는 과정에서 거리기반 알고리즘 knn을 사용하기 때문에 스케일링이 필요하다고 판단
- 이상치
  - train과 test를 통합해 이상치 처리 해준 후 다시 분할.
  - 박스플롯, IQR로 이상치 확인. But 칼럼 특성상 이상치로 판단되지 않으면 처리 X
  - 상한값, 하한값으로 대체
- 결측치: 평균, 중앙값, 최빈값 등으로 채워보는 시도 진행 ->  KNNImputer를 이용해 결측치 채우는 방식이 가장 성능이 높았음(역 채택 ...)
- 칼럼 범주 변환: Age 칼럼 범주화 진행 - 과적합을 예방하고, 모델 해석측면에서 도움이 된다고 판단해 진행.인코딩까지 진행함.
- 피처 선택:  PermutationImportance 기반으로 중요도 낮다고 판단되는 칼럼 삭제
- 불균형 데이터 처리: 불균형 레이블 확인 후 SMOTE, ADASYN 등과 같은 오버샘플링 기법들을 사용해보았지만 성능을 향상시키지 못함 -> class_weight = 'balanced' 인자로 미미하게 성능 향상시킴. (클래스별로 가중치를 다르게 부여, 각 클래스에 할당할 가중치(Weight) 계산.)


모델링
- CatBoost 이용
  - 불균형 데이터 가중치 미리 계산해둔 뒤, CatBoost 인자로 사용.
  - 교차검증(cross_val_score)로 일반화(Generalization) 능력을 측정.



# 차별점 및 배울점 : 새롭게 얻은 인사이트, 인상깊었던 점

인상깊은 점
- 결측치 유지 위해 인코딩 시 결측치 없는 칼럼과 있는 칼럼을 나누어 유지한 점.
- 결측치 처리 전 인코딩, 스케일링, 이상치 처리를 거친 뒤 최적의 값으로 결측치 채우는 접근 방식.
- 이상치를 가지는 칼럼이라도 칼럼 특성상 이상치가 문제되지 않는다면 그대로 유지한 점  
- 이상치 판단 시 train dataset에서만 판단하지 않고, train과 test data를 합쳐 샘플 수를 늘린 것.
- 과다 범주로 여겨질 수 있는 Age 칼럼을 자체 범주화 조정


새롭게 숙지한 개념
-  KNNImputer: 결측치($\text{NaN}$)가 포함된 데이터 포인트(관측치)에 대해, 데이터셋 내에서 가장 유사한 $k$개의 다른 데이터 포인트를 찾고, 결측치($\text{NaN}$)가 있는 특징(Feature)은 거리 계산에서 제외하고, 결측치가 없는 특징들만을 사용하여 거리를 계산하여 결측치($\text{NaN}$)가 포함된 데이터 포인트(관측치)에 대해, 데이터셋 내에서 가장 유사한 $k$개의 다른 데이터 포인트를 찾아 평균값으로 대체함.
